In [1]:
import os
from openai import OpenAI

In [2]:
MODEL = "gpt-3.5-turbo"
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [3]:
def get_completion(prompt, model=MODEL, temperature=0):
    messages = [
        {"role": "user", "content": f"{prompt}"}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,# degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Prompting Principles

### Principle 1: Write clear and specific instructions

##### Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [9]:
text = """
A solenoid is an electrically energized coil that forms an electromagnet capable of performing mechanical functions. The term ‘solenoid’ is derived from the word ‘sole’ which in reference to electrical equipment means ‘a part of,’ or ‘contained inside, or with, other electrical equipment.’ The Greek word solenoides means ‘Channel,’ or ‘shaped like a pipe.’ A simple plunger-type solenoid consists of a coil of wire attached to an electrical source, and an iron rod, or plunger, that passes in and out of the coil along the axis of the spiral. A return spring holds the rod outside the coil when the current is deenergized, as shown in figure 1
"""

prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""

print(get_completion(prompt))

A solenoid is an electrically energized coil that forms an electromagnet capable of performing mechanical functions by moving a plunger-type component along its axis.


##### Ask for a structured output
JSON, HTML

In [10]:
prompt = f"""
Generate a list of three books along with their authors and genre. Provide them in JSON format with keys: id, title, author, genre
"""

print(get_completion(prompt))

Here is the list of books in JSON format:

```
[
  {
    "id": 1,
    "title": "To Kill a Mockingbird",
    "author": "Harper Lee",
    "genre": "Classic Fiction"
  },
  {
    "id": 2,
    "title": "1984",
    "author": "George Orwell",
    "genre": "Dystopian Science Fiction"
  },
  {
    "id": 3,
    "title": "Pride and Prejudice",
    "author": "Jane Austen",
    "genre": "Romance Classic Fiction"
  }
]
```

Let me know if you'd like me to generate more books or change the format in any way!


##### Ask the model to check whether conditions are satisfied

In [17]:
recipe = f"""
Preheat the oven to 325 degrees F. Butter a 9 x 5 x 3 inch loaf pan.
Cream the sugar and butter in a large mixing bowl until light and fluffy. Add the eggs one at a time, beating well after each addition. In a small bowl, mash the bananas with a fork. Mix in the milk and cinnamon. In another bowl, mix together the flour, baking powder, baking soda and salt. Add the banana mixture to the creamed mixture and stir until combined. Add dry ingredients, mixing just until flour disappears. Pour batter into prepared pan and bake 1 hour to 1 hour 10 minutes, until a toothpick inserted in the center comes out clean. Set aside to cool on a rack for 15 minutes. Remove bread from pan, invert onto rack and cool completely before slicing. Spread slices with honey or serve with ice cream.
"""

In [24]:
prompt = """
Analyze the text provided below, delimited by triple backticks. If it contains sequence of instructions, provide those instructions in the following format:
step 1: ...
step 2: ...
...
step N: ...
If the text does not contain any sequence of instruction, then simply write 'No steps found'
```{text}```
"""

In [22]:
print(get_completion(prompt.format(text=recipe)))

step 1: Preheat the oven to 325 degrees F.
step 2: Butter a 9 x 5 x 3 inch loaf pan.
step 3: Cream the sugar and butter in a large mixing bowl until light and fluffy.
step 4: Add the eggs one at a time, beating well after each addition.
step 5: In a small bowl, mash the bananas with a fork.
step 6: Mix in the milk and cinnamon.
step 7: In another bowl, mix together the flour, baking powder, baking soda and salt.
step 8: Add the banana mixture to the creamed mixture and stir until combined.
step 9: Add dry ingredients, mixing just until flour disappears.
step 10: Pour batter into prepared pan and bake 1 hour to 1 hour 10 minutes, until a toothpick inserted in the center comes out clean.
step 11: Set aside to cool on a rack for 15 minutes.
step 12: Remove bread from pan, invert onto rack and cool completely before slicing.
step 13: Spread slices with honey or serve with ice cream.


In [25]:
print(get_completion(prompt.format(text=text)))

No steps found


##### Few-shot prompting

In [32]:
prompt = f"""
You are required to answer in consistent style.
Dialog:
Josh: Happy Birthday, this is for you, Brian.
Brian: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
Josh: Brian, may I have a pleasure to have a dance with you?
Brian: Ok.
Josh: This is really wonderful party.
Brian: Yes, you are always popular with everyone. and you look very pretty today.
Josh: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
Brian: You look great, you are absolutely glowing.
Josh: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Question:
What was going on?
Answer:
Josh attends Brian's birthday party. Brian thinks Josh looks great and charming.

Dialog:
Seth: Yeah. Just pull on this strip. Then peel off the back.
Dave: You might make a few enemies this way.
Seth: If they don't think this is fun, they're not meant to be our friends.
Dave: You mean your friends. I think it's cruel.
Seth: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
Dave: Hurry! Get a shot!. . . Hand it over!
Seth: I knew you'd come around. . .
Question:
What was going on?
Answer:
"""

print(get_completion(prompt))

Here are the answers in consistent style:

1. What was going on?

Josh attends Brian's birthday party, and Brian is happy to see Josh and enjoys his company.

2. What was going on?

Seth and Dave appear to be engaging in a prank or a game that involves pulling off a strip of clothing from two people, possibly as part of a dare or a challenge. The conversation suggests that Seth is enthusiastic about the activity, while Dave seems uncomfortable with it.


### Principle 2: Give the model time to “think”

##### Specify the steps required to complete a task

In [35]:
text_2 = f"""
In 1925 the land aristocracy of Germany owned most of the large estates which occupied 20.2 per cent of the arable land of the country. They had 40 per cent of the land east of the Elbe River. All told, these large estates constituted but 0.4 per cent of the total number of landholdings in Germany. At the base of the pyramid were those who occupied small holdings: 59.4 per cent of the total holdings of Germany accounted for only 6.2 per cent of the arable land.
"""

prompt = f"""
Your task is to perform the following steps:
1. Summarize the provided text delimited by triple backticks in one sentence
2. Translate the summary into Spanish
3. List down the places or locations in the summary.
4. Provide a json object with keys: spanish_summary, places, num_places
Separate answers with line break.
```{text_2}```
"""

print(get_completion(prompt))

Here is a summary of the text in one sentence:


The large estates in Germany, owned by the aristocracy, held a significant portion of the country's arable land.


Here is the translation of the summary into Spanish:


Las grandes fincas en Alemania, propiedad de la nobleza, poseían una gran parte del suelo fértil del país.


Here are the places or locations mentioned in the summary:


* Germany
* Elbe River


Here is the JSON object with keys: spanish_summary, places, num_places:


```
{
  "spanish_summary": "Las grandes fincas en Alemania, propiedad de la nobleza, poseían una gran parte del suelo fértil del país.",
  "places": ["Alemania", "Elbe River"],
  "num_places": 2
}
```


##### Ask for output in a specified format

In [37]:
text_2 = f"""
In 1925 the land aristocracy of Germany owned most of the large estates which occupied 20.2 per cent of the arable land of the country. They had 40 per cent of the land east of the Elbe River. All told, these large estates constituted but 0.4 per cent of the total number of landholdings in Germany. At the base of the pyramid were those who occupied small holdings: 59.4 per cent of the total holdings of Germany accounted for only 6.2 per cent of the arable land.
"""

prompt = f"""
Your task is to perform the following steps:
1. Summarize the provided text delimited by triple backticks in one sentence
2. Translate the summary into Spanish
3. List down the places or locations in the translation.
4. Provide a json object with keys: spanish_summary, places, num_places
Separate answers with line break. Use the following format for answer:
Text: <text_to_summarize>
Summary: <generated_summary>
Translation: <generated_translation>
Places: <list_of_places>
JSON Output: <required_json_object>
```{text_2}```
"""

print(get_completion(prompt))

Text: In 1925, the German aristocracy owned most of the large estates that occupied a significant portion of the country's arable land.

Summary: The German aristocracy owned most of the large estates in Germany in 1925.

Translation: La aristocracia alemana poseía la mayoría de las grandes fincas que ocupaban una gran parte del suelo fértil del país en 1925.

Places: Elbe River

JSON Output:
{
  "spanish_summary": "La aristocracia alemana poseía la mayoría de las grandes fincas que ocupaban una gran parte del suelo fértil del país en 1925.",
  "places": [
    "Elbe River"
  ],
  "num_places": 1
}


##### Instruct the model to work out its own solution before rushing to a conclusion

In [4]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

In [6]:
print(get_completion(prompt))

The student's solution is mostly correct.

Here are the strengths of their solution:

* They correctly identified all the costs involved in the solar power installation.
* They used a variable (x) to represent the size of the installation in square feet, which makes it easy to calculate the total cost as a function of x.
* They combined like terms and simplified the expression for the total cost.

However, there is one minor error:

The student forgot to include the maintenance cost per square foot ($10) in their calculation. The correct maintenance cost should be 100x + 10x = 110x.

Here's the corrected solution:

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 110x (not just 100,000 + 100x)
Total cost: 100x + 250x + 100,000 + 110x = 460x + 100,000

Overall, the student's solution is correct with a minor correction.


In [9]:
prompt = f"""
Your task is to determine if the student's solution
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost
me a flat $100k per year, and an additional $10 / square
foot
What is the total cost for the first year of operations
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

print(get_completion(prompt))

Question:
```
I'm building a solar power installation and I need help
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost
me a flat $100k per year, and an additional $10 / square
foot
What is the total cost for the first year of operations
as a function of the number of square feet.
```

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```

Actual solution:
To calculate the total cost for the first year of operations as a function of the number of square feet, we need to add up all the costs.

1. Land cost: $100 per square foot, so the land cost is $100x.
2. Solar panel cost: $250 per square foot, so the solar panel cost is $250x.
3. Maintenance cost: The maintenance contract costs a flat $100k per year,